# Python 3.10: Cool New Features for You to Try

## Better Error Messages

Run the examples:

- `rp310_hello.py`
- `rp310_unterminated_dict.py`
- `rp310_missing_comma.py`
- `rp310_assignment_equality.py`
- `rp310_misspell_1.py`
- `rp310_misspell_2.py`
- `rp310_misspell_3.py`